# Signal Induced Noise Analysis

### Introduction
<p style='text-align: justify'>
The Signal Induced Noise (SIN) measurement is used to quantify the constitution of after-pulses with respect to a true signal. After calculating the integrated signal and the integrated SIN, the SIN Fraction can be calculated using the following formula:
$$SIN_{fraction}=\frac{noise}{signal+noise}$$
together with the Signal to Noise (S/N) ratio:
$$SN_{ratio} =\frac{signal}{noise}$$
Both parameters provide an information of the intensity of the SIN to the signal.
</p>